In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

from lightgbm import LGBMClassifier

## mutual information

In [7]:
def get_mi(a, b, c = None):
    if a.ndim == 1:
        a = a[:, None]
    if b.ndim == 1:
        b = b[:, None]
        
    if c is None:
        c = np.ones([len(a), 1]) * 0.5
    if c.ndim == 1:
        c = c[:, None]
        
    """
    a : feature : (n_data, a_dim)
    b : target : (n_data, b_dim)
    c : conditional feature : (n_data, c_dim)
    """

    a = a.reshape(a.shape[0], a.shape[1], 1, 1, 1)
    b = b.reshape(b.shape[0], 1, b.shape[1], 1, 1)
    c = c.reshape(c.shape[0], c.shape[1], 1)
    # a.shape = (n_data, a_dim, b_dim, a_onehot, b_onehot)
    # b.shape = (n_data, a_dim, b_dim, a_onehot, b_onehot)
    # c.shape = (n_data, c_dim, c_onehot)

    ## bool -> onehot
    a = np.concatenate([1 - a, a], axis = 3)
    b = np.concatenate([1 - b, b], axis = 4)
    c = np.concatenate([1 - c, c], axis = 2)

    ## marginal probability
    abc = np.tensordot(a * b, c, axes = [0,0])
    pabc = abc / np.sum(np.sum(abc, axis = 2, keepdims = True), axis = 3, keepdims = True) + 1e-16
    pa = np.sum(pabc, axis = 3, keepdims=True)
    pb = np.sum(pabc, axis = 2, keepdims=True)

    ## mutual information
    mi = np.sum((np.sum(np.sum(pabc * np.log(pabc / pa / pb), axis = 2), axis = 2) *  np.mean(c, axis = 0)[None, None, :]), axis = -1)
    
    """
    mi.shape = (a_dim, b_dim, c_dim)
    """
    return mi

In [8]:
n_data = 10000
x = np.random.binomial(1, 0.5, size = n_data)
z = np.random.binomial(1, 0.5, size = n_data)
y = x.copy()
y[z == 1] = 1 - y[z == 1] ## xだけ見るとyと関係はないが，xとzを見るとyはわかる

In [9]:
## xとyには一見関係がない -> xはy予測の特徴量としては微妙そうに見える -> mutual informationは小さい
get_mi(x, y)

array([[[0.00010737]]])

In [10]:
## zで条件付けるとxとyには関係がある -> xとzのinteractionは特徴量として効きそう -> conditional mutual informationは大きい
get_mi(x, y, z)

array([[[0.69286808]]])

## シミュレーション

In [18]:
## interactionが重要になる二値分類データを作る

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

np.random.seed(42 + 1)

## データ設定
n_dim = 50
n_data = 10000
int_cols = [[1, 2, 3, 4, 5], [6, 7, 8, 9], [10, 11, 12], [13, 14]]

## データ生成
x = np.random.binomial(1, 0.5, [n_data, n_dim])
fs = []
for k in range(len(int_cols)):
    int_col = int_cols[k]
    f = 1
    for i in range(len(int_col)):
        f *= (x[:, int_col[i]] + i / 5) ## binaryになんか足してかけていく
    
    for i in range(1, len(int_col)):
        reverse = x[:, int_col[i]]
        f[reverse == 1] = -f[reverse == 1] ## binaryが0か1かで正負が入れ替わる interactionを作る
    fs.append(f)
    
f = np.vstack(fs).T
w = np.random.randn(f.shape[1])
logit = np.sum(f * w, axis = 1)
p = sigmoid(logit)
y = np.random.binomial(1, p)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True, random_state = 42)

In [20]:
def eval_linear(x_train, x_test):
    model = LogisticRegression(solver = "lbfgs")
    model.fit(x_train, y_train)
    p = model.predict_proba(x_test)[:,1]
    print("auc : ", roc_auc_score(y_test, p))

### 生の特徴量をそのままlinearに

In [21]:
# raw
eval_linear(x_train, x_test)

auc :  0.5629372203365415


## CMIを元にinteraction特徴量を作る

In [22]:
def get_interaction_by_cmi_top(x1_train, x2_train, x1_test, x2_test, y_train, n = 10):
#     x1_train, x1_test, x2_train, x2_test, y_train, y_test = train_test_split(x1, x2, y, test_size = 0.2, shuffle = True, random_state = 42)
    
    mi = get_mi(x1_train, y_train)
    cmi = get_mi(x1_train, y_train, x2_train)
    
#     mi = get_mi(x1_train[:10000], y_train[:10000])
#     cmi = get_mi(x1_train[:10000], y_train[:10000], x2_train[:10000])
    
    score = cmi - mi
    score = score[:,0,:] ## targetは１次元と仮定

    idx = np.argsort(score.flatten())[::-1][:n]
    idx1 = idx // score.shape[1]
    idx2 = idx % score.shape[1]

    x1 = np.vstack([x1_train, x1_test])
    x2 = np.vstack([x2_train, x2_test])
    
    f = []
    for j in range(n):
        interection = x1[:,idx1[j]].astype(str).astype("object") + "_" + x2[:,idx2[j]].astype(str).astype("object")
        interection_oh = pd.get_dummies(interection).values
        f.append(interection_oh)
    f = np.hstack(f)
    return f[:len(x1_train)], f[len(x1_train):], score


In [24]:
## インタラクション（CMIを用いた選択，２次まで）
f_train, f_test, score = get_interaction_by_cmi_top(x_train, x_train, x_test, x_test, y_train, n = 50)
f_train = np.hstack([f_train, x_train])
f_test = np.hstack([f_test, x_test])

eval_linear(f_train, f_test)

auc :  0.6076163495021702


In [25]:
## インタラクション（CMIを用いた選択，３次まで）
f1_train, f1_test, score = get_interaction_by_cmi_top(x_train, x_train, x_test, x_test, y_train, n = 50)
f2_train, f2_test, score = get_interaction_by_cmi_top(x_train, f1_train, x_test, f1_test, y_train, n = 30)
f_train = np.hstack([f1_train, f2_train, x_train])
f_test = np.hstack([f1_test, f2_test, x_test])
eval_linear(f_train, f_test)

auc :  0.6542101698713236


In [26]:
## インタラクション（CMIを用いた選択，４次まで）
f1_train, f1_test, score = get_interaction_by_cmi_top(x_train, x_train, x_test, x_test, y_train, n = 50)
f2_train, f2_test, score = get_interaction_by_cmi_top(x_train, f1_train, x_test, f1_test, y_train, n = 30)
f3_train, f3_test, score = get_interaction_by_cmi_top(x_train, f2_train, x_test, f2_test, y_train, n = 20)
f_train = np.hstack([f1_train, f2_train, f3_train, x_train])
f_test = np.hstack([f1_test, f2_test, f3_test, x_test])
eval_linear(f_train, f_test)

auc :  0.6846261500401645


In [27]:
## インタラクション（CMIを用いた選択，５次まで）
f1_train, f1_test, score = get_interaction_by_cmi_top(x_train, x_train, x_test, x_test, y_train, n = 50)
f2_train, f2_test, score = get_interaction_by_cmi_top(x_train, f1_train, x_test, f1_test, y_train, n = 30)
f3_train, f3_test, score = get_interaction_by_cmi_top(x_train, f2_train, x_test, f2_test, y_train, n = 20)
f4_train, f4_test, score = get_interaction_by_cmi_top(x_train, f3_train, x_test, f3_test, y_train, n = 20)
f_train = np.hstack([f1_train, f2_train, f3_train, f4_train, x_train])
f_test = np.hstack([f1_test, f2_test, f3_test, f4_test, x_test])
eval_linear(f_train, f_test)

auc :  0.6922418993256565


## 上までで作った特徴量をlgbに食わせる

In [31]:
##### lgbm CMI ５次まで
model = LGBMClassifier()
model.fit(f_train, y_train)
p = model.predict_proba(f_test)[:,1]
print("auc : ", roc_auc_score(y_test, p))

auc :  0.7312919964107043


In [32]:
## 元々の特徴量をlgbに食わせたら
model = LGBMClassifier()
model.fit(x_train, y_train)
p = model.predict_proba(x_test)[:,1]
print("auc : ", roc_auc_score(y_test, p))

auc :  0.6603683929898693


## 結論

## lgb with interaction > linear with interaction > lgb wth raw > linear with raw